In [11]:
pwd

'/content'

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WIb_ACjnfg57VW8XciXcPyyAXvbD1wVj0BCjSLJ4vB_X',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-udm84rrbrf7tlw'
object_key = 'TRAIN_SET.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [ ]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [10]:
ls

sample_data/


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of Layers
from tensorflow.keras import layers #A Layer consists of a tensor-in tensor-out computation function
#Dense Layer is the regular deeply connected neural network Layer
from tensorflow.keras.layers import Dense, Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
x_train = train_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 2626 images belonging to 5 classes.

In [ ]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}

In [ ]:
from collections import Counter as c 
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [ ]:
# Initializing the CNN classifier = Sequential()
classifier = Sequential()
# First convolution layer and pooling
classifier.add(Conv2D(32,(3, 3), input_shape=(64, 64, 3),activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the Layers
classifier.add(Flatten())
# Adding fully connected Layer a
classifier.add(Dense (units=128, activation='relu'))
classifier.add(Dense (units=5, activation='softmax')) # softmax for more than 2

In [ ]:
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               802944    
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
=================================================================
Total params: 813,733
Trainable params: 813,733
Non-trainable params: 0
_________________________________________________________________

In [ ]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
##Fitting the model
classifier.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train), 
    epochs=20,validation_data=x_test,validation_steps = len(x_test)) # No of images in test set

Epoch 1/20
526/526 [==============================] - 15s 27ms/step - loss: 0.1318 - accuracy: 0.9524 - val_loss: 1.6592e-04 - val_accuracy: 1.0000
Epoch 2/20
526/526 [==============================] - 14s 27ms/step - loss: 0.0206 - accuracy: 0.9950 - val_loss: 0.4374 - val_accuracy: 0.7871
Epoch 3/20
526/526 [==============================] - 14s 27ms/step - loss: 0.0325 - accuracy: 0.9897 - val_loss: 8.4484e-05 - val_accuracy: 1.0000
Epoch 4/20
526/526 [==============================] - 14s 27ms/step - loss: 1.2242e-04 - accuracy: 1.0000 - val_loss: 1.7502e-05 - val_accuracy: 1.0000
Epoch 5/20
526/526 [==============================] - 14s 27ms/step - loss: 3.8190e-05 - accuracy: 1.0000 - val_loss: 1.0376e-05 - val_accuracy: 1.0000
Epoch 6/20
526/526 [==============================] - 14s 27ms/step - loss: 2.1320e-05 - accuracy: 1.0000 - val_loss: 4.5576e-06 - val_accuracy: 1.0000
Epoch 7/20
526/526 [==============================] - 14s 27ms/step - loss: 8.8509e-06 - accuracy: 1.0000 - val_loss: 2.7205e-06 - val_accuracy: 1.0000
Epoch 8/20
526/526 [==============================] - 14s 27ms/step - loss: 7.3976e-06 - accuracy: 1.0000 - val_loss: 1.5667e-06 - val_accuracy: 1.0000
Epoch 9/20
526/526 [==============================] - 14s 26ms/step - loss: 5.0094e-06 - accuracy: 1.0000 - val_loss: 9.0818e-07 - val_accuracy: 1.0000
Epoch 10/20
526/526 [==============================] - 14s 27ms/step - loss: 2.3061e-06 - accuracy: 1.0000 - val_loss: 6.9995e-07 - val_accuracy: 1.0000
Epoch 11/20
526/526 [==============================] - 14s 26ms/step - loss: 2.4215e-06 - accuracy: 1.0000 - val_loss: 5.5605e-07 - val_accuracy: 1.0000
Epoch 12/20
526/526 [==============================] - 14s 27ms/step - loss: 1.2473e-06 - accuracy: 1.0000 - val_loss: 4.2676e-07 - val_accuracy: 1.0000
Epoch 13/20
526/526 [==============================] - 14s 27ms/step - loss: 8.6773e-07 - accuracy: 1.0000 - val_loss: 3.2626e-07 - val_accuracy: 1.0000
Epoch 14/20
526/526 [==============================] - 14s 27ms/step - loss: 2.0117e-06 - accuracy: 1.0000 - val_loss: 1.9171e-07 - val_accuracy: 1.0000
Epoch 15/20
526/526 [==============================] - 14s 27ms/step - loss: 4.7506e-07 - accuracy: 1.0000 - val_loss: 1.0427e-07 - val_accuracy: 1.0000
Epoch 16/20
526/526 [==============================] - 14s 27ms/step - loss: 3.3093e-07 - accuracy: 1.0000 - val_loss: 7.6310e-08 - val_accuracy: 1.0000
Epoch 17/20
526/526 [==============================] - 14s 27ms/step - loss: 2.6706e-07 - accuracy: 1.0000 - val_loss: 4.1174e-08 - val_accuracy: 1.0000
Epoch 18/20
526/526 [==============================] - 14s 27ms/step - loss: 2.3379e-07 - accuracy: 1.0000 - val_loss: 4.5804e-08 - val_accuracy: 1.0000
Epoch 19/20
526/526 [==============================] - 14s 27ms/step - loss: 2.2811e-07 - accuracy: 1.0000 - val_loss: 2.8781e-08 - val_accuracy: 1.0000
Epoch 20/20
526/526 [==============================] - 14s 27ms/step - loss: 1.1122e-07 - accuracy: 1.0000 - val_loss: 1.3437e-08 - val_accuracy: 1.0000

In [7]:
ls

sample_data/


In [ ]:
classifier.save('nutrition.h5')

In [ ]:
ls

Dataset/  nutrition.h5  TRAIN_SET/

In [6]:
pwd

'/content'

In [ ]:
### Predicting our results
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition.h5") #Loading the model for testing
from tensorflow.keras.preprocessing import image

In [ ]:
img =image.load_img(r"/home/wsuser/work/TRAIN_SET/ORANGE/100_100.jpg",grayscale=False, target_size= (64,64))
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis=0) #changing the shape =
y =np.argmax(model.predict(x),axis=1)
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
index[y[0]]

'ORANGE'

IBM Deployment

In [5]:
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 538 kB 33.9 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 132 kB 71.0 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 9.9 MB 50.7 MB/s 
     |████████████████████████████████| 140 kB 67.3 MB/s 
     |████████████████████████████████| 956 kB 55.1 MB/s 
     |████████████████████████████████| 135 kB 70.3 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.12.0-py3-none-any.whl size=73931 sha256=32501973f32267aa4bcad03662d33887ab744b1750f81bcbb2e2c19f9420e66f
  Stored in directory: /root/.cache/pip/wheels/ec/94/29/2b57327cf00664b6614304f7958abd29d77ea0e5bbece2ea57
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.12.0-py3-none-any.whl size=562962 sha256=31e5319d44

In [1]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 312 kB 31.9 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.2.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 454.3 MB 19 kB/s 
     |████████████████████████████████| 14.8 MB 49.6 MB/s 
     |████████████████████████████████| 1.2 MB 71.1 MB/s 
     |████████████████████████████████| 462 kB 82.1 MB/s 
     |███████████████████████████

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey": "mvD9uH2XT7dZZf-7SAS7hLPBA8AfBd9pCSzGNIsC26uO"
}

client=APIClient(wml_credentials)

In [ ]:
client

In [ ]:
def guid_space_name(client,nutrition_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==nutrition_deploy)['metadata']['id'])

Deployment

In [ ]:
space_uid=guid_space_name(client,'nutrition_deploy')
print("Space UID " + space_uid)

Space UID ef027b06-f8be-4794-a8d8-78f31828274d

In [ ]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list(200)

In [ ]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [ ]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
!tar -zcvf nutrition-classification-model.tgz nutrition.h5

nutrition.h5

In [ ]:
model_details=client.repository.store_model(model ='nutrition-classification-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN Model Building",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_uid
})

In [ ]:
model_id=client.repository.get_model_id(model_details)

In [ ]:
model_id

'cfd303f2-c0bc-44b4-b58b-52def6a33906'

In [ ]:
ls

Dataset/  nutrition-classification-model.tgz  nutrition.h5  TRAIN_SET/

In [ ]:
client.repository.download(model_id,'nutrition.tar.gb')

Successfully saved model content to file: 'nutrition.tar.gb'
'/home/wsuser/work/nutrition.tar.gb'

In [ ]:
pwd

'/home/wsuser/work'